In [1]:
import torch
from torch import nn
import csv
import os
import re
from collections import Counter
import numpy as np

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'airline_intent_dataset.csv',
 'Bot.ipynb',
 'bot.pt',
 'cb_dataset_cleaned - Sheet1.csv',
 'chatbot_1_model.ipynb',
 'data',
 'drop_dataset',
 'drop_dataset.zip',
 'embed.npy',
 'embed.pt',
 'flight_a.txt',
 'flight_data.ipynb',
 'flight_q.txt',
 'gg.ipynb',
 'good_one.json',
 'home',
 'intent.txt',
 'intent_int.txt',
 'int_to_vocab.txt',
 'opencon_dataset_augmented2.csv',
 'open_a.txt',
 'Open_con_Dataset.ipynb',
 'open_q.txt',
 'preprocessing_chat_bot.ipynb',
 'preprocessing_chat_bot.ipynb - Shortcut.lnk',
 'preprocessing_chat_bot.py',
 'sentrence.txt',
 'sentrence_int.txt',
 'skipgram.pth',
 'skipgram_embed_intent_bot.pt',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'utils.py',
 'vocab_to_int.txt',
 'weight.pt',
 'workspace-1607731597.tar.gz',
 '__pycache__']

In [3]:
f=open('opencon_dataset_augmented2.csv') 
data=csv.reader(f)

In [4]:
question=[]
answer=[]
for i in data:
    question.append(i[0])
    answer.append(i[1])

In [58]:
answer[:20]

['GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot',
 'GQ.bot']

In [59]:
question[:20]

['Is there a bot chatting to me?',
 'Is it automated message?',
 'Computer based pely',
 'Bot or human?',
 'Bot is chatting with me?',
 'Are you system generated message?',
 'Are you robot?',
 'Are you machine?',
 'Are you just computer?',
 'Are you a robot?',
 'Are you a person?',
 'Are you a machine?',
 'Are you a chatbot?',
 'Are you a bot?',
 'Are these automated messages?',
 'Am I talking to a bot?',
 'You are a boy are girl?',
 'You are a bot or human?',
 'This is a machine?',
 'This is a chatbot?']

In [60]:
intent=list(set(answer))
intent

['FAQ.accom',
 'SQ.event_prize',
 'SQ.event_speakers',
 'GQ.help',
 'GQ.bot',
 'GQ.name',
 'FAQ.contact_info',
 'JOIN.sponsor',
 'SQ.event_details',
 'SQ.reg_fee',
 'FAQ.food',
 'SQ.reg_lastdate',
 'FAQ.why_reg',
 'SQ.event_schedule',
 'JOIN.speaker',
 'GQ.gen',
 'SQ.IEEE',
 'GQ.query',
 'SQ.event_date']

In [14]:
intent_int={}
int_intent={}
for i,j in enumerate(intent):
    intent_int.update({j:i})
for i,j in intent_int.items():
    int_intent.update({i:j})
    

In [61]:
def preproces_1(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words


def preproces_2(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 0]

    return trimmed_words

def create_lookup_tables(words):
    word_counts = Counter(words)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab


In [62]:
preproces_2(question[1])

['is', 'it', 'automated', 'message', '<QUESTION_MARK>']

In [ ]:
question_p=[]
for i in question:
    sent=' '.join(preproces_2(i))+"\n"
    question_p.append(sent)

In [26]:
words=[]
for i in question_p:
    for j in i.split():
        words.append(j)

In [27]:
words=list(set(words))

In [28]:
words

['interested',
 'covered',
 'below',
 'keep',
 'un',
 '2016',
 'number',
 'march',
 'somehow',
 'article',
 'named',
 'placed',
 'let',
 'dress',
 'host',
 'intended',
 'annual',
 '2020',
 'planned',
 'he',
 'domains',
 'financial',
 'themselves',
 'provide',
 'forum',
 'runner',
 'supposedly',
 'junior',
 'upcoming',
 'personal',
 "what'he",
 'clearly',
 'prior',
 'cutting',
 'legally',
 'wall',
 'backed',
 'notice',
 'formally',
 'held',
 'again',
 'wide',
 'wondered',
 'lose',
 'beyond',
 'finals',
 'approach',
 '12',
 'okay',
 'fee',
 'unique',
 'town',
 'others',
 'colour',
 'restaurant',
 'comfort',
 'right',
 'with',
 'besides',
 'losing',
 'however',
 'away',
 'adding',
 'just',
 'numbers',
 'reach',
 'somewhere',
 'please',
 'four',
 'insurance',
 'guests',
 'automated',
 'special',
 "what'las",
 'side',
 'an',
 'glad',
 'to',
 'their',
 'queries',
 '7',
 '1',
 'tell',
 'something',
 'performing',
 'successfully',
 'using',
 'outside',
 'organic',
 'enjoying',
 'remotely',
 'm

In [65]:
len(words)

1410

In [64]:
question_p[:20]

['is there a bot chatting to me <QUESTION_MARK>\n',
 'is it automated message <QUESTION_MARK>\n',
 'computer based pely\n',
 'bot or human <QUESTION_MARK>\n',
 'bot is chatting with me <QUESTION_MARK>\n',
 'are you system generated message <QUESTION_MARK>\n',
 'are you robot <QUESTION_MARK>\n',
 'are you machine <QUESTION_MARK>\n',
 'are you just computer <QUESTION_MARK>\n',
 'are you a robot <QUESTION_MARK>\n',
 'are you a person <QUESTION_MARK>\n',
 'are you a machine <QUESTION_MARK>\n',
 'are you a chatbot <QUESTION_MARK>\n',
 'are you a bot <QUESTION_MARK>\n',
 'are these automated messages <QUESTION_MARK>\n',
 'am i talking to a bot <QUESTION_MARK>\n',
 'you are a boy are girl <QUESTION_MARK>\n',
 'you are a bot or human <QUESTION_MARK>\n',
 'this is a machine <QUESTION_MARK>\n',
 'this is a chatbot <QUESTION_MARK>\n']

In [33]:
words=list(set(words))
int_to_words={}
words_to_int={}
for i,j in enumerate(words,1):
    int_to_words.update({i:j})
for i,j in int_to_words.items():
    words_to_int.update({j:i})

In [35]:
questions_int=[]
for i in question_p:
    question_int=[]
    for j in i.split():
        if j in words:
            question_int.append(words_to_int[j])
    if len(question_int):    
        questions_int.append(question_int)
    

In [36]:
questions_int

[[1187, 781, 715, 1184, 485, 78, 194, 216],
 [1187, 704, 72, 1169, 216],
 [1209, 663, 879],
 [1184, 822, 1213, 216],
 [1184, 1187, 485, 58, 194, 216],
 [247, 1391, 220, 1073, 1169, 216],
 [247, 1391, 1339, 216],
 [247, 1391, 883, 216],
 [247, 1391, 64, 1209, 216],
 [247, 1391, 715, 1339, 216],
 [247, 1391, 715, 1031, 216],
 [247, 1391, 715, 883, 216],
 [247, 1391, 715, 784, 216],
 [247, 1391, 715, 1184, 216],
 [247, 484, 72, 933, 216],
 [1222, 763, 1136, 78, 715, 1184, 216],
 [1391, 247, 715, 554, 247, 424, 216],
 [1391, 247, 715, 1184, 822, 1213, 216],
 [431, 1187, 715, 883, 216],
 [431, 1187, 715, 784, 216],
 [220, 1073, 425],
 [457, 1391, 220, 637],
 [1187, 431, 1209, 663, 1035],
 [1187, 431, 715, 883, 216],
 [247, 1391, 715, 1213, 216],
 [759, 1165, 68],
 [759, 1165],
 [763, 759, 1125, 644],
 [1160, 1391, 1165, 194, 216],
 [522, 809, 867, 1391, 595, 216],
 [522, 595, 1391, 595, 216],
 [522, 867, 1391, 1165, 194, 58, 216],
 [522, 867, 1391, 595, 216],
 [522, 247, 1391, 562, 631, 216

In [37]:
def pad_features(reviews_ints, seq_length):

    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [38]:
questions_int=np.array(questions_int)

C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [39]:
seq_length = 10

features = pad_features(questions_int, seq_length=seq_length)

## test statements - do not change - ##
#assert len(features)==len(sentrences.split('\n')[:20]), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features)

[[   0    0 1187 ...   78  194  216]
 [   0    0    0 ...   72 1169  216]
 [   0    0    0 ... 1209  663  879]
 ...
 [1207   78  959 ...  800  431  611]
 [ 522 1041  522 ... 1168 1391  759]
 [ 207  522 1187 ...  193  203  203]]


In [40]:
ans_set=list(set(answer))
ans_to_int={}
int_to_ans={}
for i,j in enumerate(ans_set):
    int_to_ans.update({i:j})
for i,j in int_to_ans.items():
    ans_to_int.update({j:i})
master_intent=[]
for i in answer:
    master_intent.append(ans_to_int[i])


In [67]:
master_intent[:40]

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]

In [42]:
split_frac = 0.9

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = master_intent[:split_idx], master_intent[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(3263, 10) 
Validation set: 	(181, 10) 
Test set: 		(182, 10)


In [55]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.Tensor(train_x), torch.Tensor(train_y))
valid_data = TensorDataset(torch.Tensor(val_x), torch.Tensor(val_y))
test_data = TensorDataset(torch.Tensor(test_x), torch.Tensor(test_y))

# dataloaders
batch_size = 1

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [44]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([1, 10])
Sample input: 
 tensor([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,   20., 1010.,  216.]])

Sample label size:  torch.Size([1])
Sample label: 
 tensor([15.])


In [45]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [68]:
import torch.nn as nn

class Semantic_Classifier(nn.Module):
   

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(Semantic_Classifier, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim*2, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
       
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        out = self.fc(torch.cat((torch.squeeze(hidden[0]),torch.squeeze(hidden[1])),0))
       
        sig_out = self.sig(out)
        sig_out = sig_out.view(batch_size, -1)
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [48]:
vocab_size = len(words_to_int)+1
output_size = 19
embedding_dim = 300
hidden_dim = 256
n_layers = 1

# net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
net_1=Semantic_Classifier(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net_1)

Semantic_Classifier(
  (embedding): Embedding(1411, 300)
  (lstm): LSTM(300, 256, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=19, bias=True)
  (sig): Sigmoid()
)


In [49]:
lr=0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net_1.parameters(), lr=lr)

In [51]:
epochs = 10
counter = 0
print_every = 1000
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net_1.cuda()

net_1.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    net_1.train()
    h = net_1.init_hidden(batch_size)
    

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs=torch.tensor(inputs).to(torch.int64)
            labels=torch.tensor(labels).to(torch.long)

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net_1.zero_grad()

        # get the output from the model
        net_1.train()
        output, h = net_1(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net_1.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            net_1.eval()
            val_h = net_1.init_hidden(batch_size)
            val_losses = []
            a=[]
            net_1.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                    inputs=torch.tensor(inputs).to(torch.int64)
                    labels=torch.tensor(labels).to(torch.long)

                output, val_h = net_1(inputs, val_h)
                _,index=torch.topk(output,1)
                val_loss = criterion(output, labels)
                a.append(index.item()==labels.item())
            a=100*((a.count(1))/(a.count(0)+a.count(1)))
                

            val_losses.append(val_loss.item())
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.9f}...".format(loss.item()),
                  "Val Accuracy: {:.2f}%...".format(a),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-pac

Epoch: 1/10... Step: 1000... Loss: 2.031302214... Val Accuracy: 96.13%... Val Loss: 2.031017
Epoch: 1/10... Step: 2000... Loss: 2.031016588... Val Accuracy: 94.48%... Val Loss: 2.031022
Epoch: 1/10... Step: 3000... Loss: 2.033260107... Val Accuracy: 94.48%... Val Loss: 2.031017
Epoch: 2/10... Step: 4000... Loss: 2.031016588... Val Accuracy: 95.03%... Val Loss: 2.031021
Epoch: 2/10... Step: 5000... Loss: 2.031016588... Val Accuracy: 95.58%... Val Loss: 2.031018
Epoch: 2/10... Step: 6000... Loss: 2.031016588... Val Accuracy: 94.48%... Val Loss: 2.044963
Epoch: 3/10... Step: 7000... Loss: 2.031016588... Val Accuracy: 94.48%... Val Loss: 2.055459
Epoch: 3/10... Step: 8000... Loss: 2.031019688... Val Accuracy: 94.48%... Val Loss: 2.031017
Epoch: 3/10... Step: 9000... Loss: 2.031021595... Val Accuracy: 95.03%... Val Loss: 2.031017
Epoch: 4/10... Step: 10000... Loss: 2.110691547... Val Accuracy: 93.92%... Val Loss: 2.031029
Epoch: 4/10... Step: 11000... Loss: 2.031021118... Val Accuracy: 95.0

In [56]:
net_1.eval()
val_h = net_1.init_hidden(batch_size)
val_losses = []
a=[]
net_1.eval()
for inputs, labels in test_loader:
        val_h = tuple([each.data for each in val_h])

        if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()
                inputs=torch.tensor(inputs).to(torch.int64)
                labels=torch.tensor(labels).to(torch.long)
                
        output, val_h = net_1(inputs, val_h)
        _,index=torch.topk(output,1)
        a.append(index.item()==labels.item())
        val_loss = criterion(output, labels)
        for i in inputs.squeeze():
            if i:
                print(int_to_words[i.item()],end=" ")
        print('OUTPUT:',int_to_ans[index.item()])
                
        
        
        val_losses.append(val_loss.item())
print( "Val Loss: {:.6f}".format(np.mean(val_losses)),"        Val_Acuuracy:",(a.count(1)/(a.count(0)+a.count(1)))*100)

C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


what do you need tell him more funds <QUESTION_MARK> OUTPUT: JOIN.sponsor
will you you always feed us <QUESTION_MARK> OUTPUT: FAQ.food
will that be we be the provided with food during OUTPUT: FAQ.food
who other should i talk to to OUTPUT: FAQ.contact_info
and when can do i speak at <PERIOD> <PERIOD> <PERIOD> OUTPUT: JOIN.speaker
but i am just a terrible family foodie <EXCLAMATION_MARK> will OUTPUT: FAQ.food
so if is there a place appropriate for speaker in OUTPUT: JOIN.speaker
what though what can fields do all things you need OUTPUT: JOIN.speaker
why are ya we gonna be provided such food OUTPUT: FAQ.food
when do you have long enough for funding <QUESTION_MARK> OUTPUT: JOIN.sponsor
such as would accommodation be on free charge or paid OUTPUT: FAQ.accom
as will the food would be better provided <QUESTION_MARK> OUTPUT: FAQ.food
i would also like to deliver us a long oral OUTPUT: JOIN.speaker
how to eventually do i apply to be this almost OUTPUT: JOIN.speaker
how much can today we apply f

how then can i help someone with the event OUTPUT: JOIN.sponsor
tell them whom to contact <QUESTION_MARK> OUTPUT: FAQ.contact_info
to where can should i stay <QUESTION_MARK> OUTPUT: FAQ.accom
of what number is the menu <QUESTION_MARK> OUTPUT: FAQ.food
because how many do know we now have to pay OUTPUT: SQ.reg_fee
what point is the procedure to finally actually become a OUTPUT: JOIN.speaker
say what are th end arrangements u for food OUTPUT: FAQ.food
so i want to <PERIOD> <PERIOD> <PERIOD> speak OUTPUT: JOIN.speaker
but what else do we now achieve just by the OUTPUT: FAQ.why_reg
had i interested in sponsoring him quite the event OUTPUT: JOIN.sponsor
give me set up the contact info of meeting someone OUTPUT: FAQ.contact_info
how many can only i speak please at your event OUTPUT: JOIN.speaker
then why should i dare come <QUESTION_MARK> OUTPUT: FAQ.why_reg
who should would i possibly contact for accommodation from health OUTPUT: FAQ.contact_info
for how much can someone truly be a bad spon

In [57]:
net_1.eval()
val_h = net_1.init_hidden(batch_size)
val_losses = []
a=[]
net_1.eval()
for inputs, labels in valid_loader:
        val_h = tuple([each.data for each in val_h])

        if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()
                inputs=torch.tensor(inputs).to(torch.int64)
                labels=torch.tensor(labels).to(torch.long)
                
        output, val_h = net_1(inputs, val_h)
        _,index=torch.topk(output,1)
        a.append(index.item()==labels.item())
        val_loss = criterion(output, labels)
        for i in inputs.squeeze():
            if i:
                print(int_to_words[i.item()],end=" ")
        print('OUTPUT:',int_to_ans[index.item()])
                
        
        
        val_losses.append(val_loss.item())
print( "Val Loss: {:.6f}".format(np.mean(val_losses)),"        Val_Acuuracy:",(a.count(1)/(a.count(0)+a.count(1)))*100)

C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Guru Aathavan AL U\code\Ana_conda\envs\pytorch_cuda_trail\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


perhaps but what is the reward <QUESTION_MARK> OUTPUT: SQ.event_prize
perhaps what will ever i get <QUESTION_MARK> OUTPUT: SQ.event_prize
one when does the loop even start of and and OUTPUT: SQ.event_date
besides when will ever the speakers not actually be coming OUTPUT: SQ.event_speakers
<PERIOD> <PERIOD> <PERIOD> what's the proper event time and have OUTPUT: SQ.event_date
now when when is it <QUESTION_MARK> OUTPUT: SQ.event_date
and will the only prize awarded be from cash or OUTPUT: SQ.event_prize
what is the maximum planned time release this span allowed OUTPUT: SQ.event_date
what or other domains / other specializations else are all OUTPUT: SQ.event_speakers
then what whatas there were in the time table for OUTPUT: SQ.event_schedule
who are the new main speakers <QUESTION_MARK> OUTPUT: SQ.event_speakers
but do the normal people running away who maybe didn't OUTPUT: SQ.event_prize
no internal speakers <QUESTION_MARK> OUTPUT: SQ.event_speakers
what is against the current last regist

when is even even the same event going by to OUTPUT: SQ.event_date
yet what is open the con <QUESTION_MARK> OUTPUT: SQ.event_details
where will help we still get t - face shirts OUTPUT: SQ.event_prize
do <PERIOD> <PERIOD> <PERIOD> you actually have any outstanding prizes OUTPUT: SQ.event_prize
when of now is it OUTPUT: SQ.event_date
only later can we then approach anyone for registration any OUTPUT: SQ.reg_lastdate
it just what does it entail OUTPUT: SQ.event_details
when of and where it is all was of it OUTPUT: SQ.event_date
on what precisely those specified dates will the public sporting OUTPUT: SQ.event_date
what is en vogue opencon <QUESTION_MARK> OUTPUT: SQ.event_details
what good is timing of already winning the event OUTPUT: SQ.event_schedule
what natural one would that be the proper last service OUTPUT: SQ.reg_lastdate
could you please name the latest speakers from who now OUTPUT: SQ.event_speakers
then till what proposed date can i apply you for OUTPUT: SQ.reg_lastdate
now wha

In [69]:
net_1

Semantic_Classifier(
  (embedding): Embedding(1411, 300)
  (lstm): LSTM(300, 256, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=19, bias=True)
  (sig): Sigmoid()
)

In [70]:
torch.save(net_1.state_dict(),"net_1.pt")